In [4]:
!pip install "paddleocr>=2.0.1"
!pip install paddlepaddle
import cv2
import numpy as np
import imutils
import re
import json
from paddleocr import PaddleOCR
ocr = PaddleOCR(use_angle_cls=True, lang='en')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 411 kB 14.0 MB/s 
     |████████████████████████████████| 604 kB 63.9 MB/s 
     |████████████████████████████████| 8.8 MB 50.9 MB/s 
     |████████████████████████████████| 148 kB 78.0 MB/s 
     |████████████████████████████████| 87 kB 7.4 MB/s 
     |████████████████████████████████| 2.2 MB 41.1 MB/s 
     |████████████████████████████████| 965 kB 61.9 MB/s 
     |████████████████████████████████| 5.6 MB 44.4 MB/s 
     |████████████████████████████████| 4.9 MB 40.5 MB/s 
     |████████████████████████████████| 399 kB 76.7 MB/s 
     |████████████████████████████████| 204 kB 74.2 MB/s 
     |████████████████████████████████| 115 kB 72.6 MB/s 
     |████████████████████████████████| 2.3 MB 58.2 MB/s 
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115943 sha256=f82076e80d9753b095fa27398ca2c7303b90997f47f31e4397ff471ee95c7d5

100%|██████████| 4.00M/4.00M [00:09<00:00, 432kiB/s] 


download https://paddleocr.bj.bcebos.com/PP-OCRv3/english/en_PP-OCRv3_rec_infer.tar to /root/.paddleocr/whl/rec/en/en_PP-OCRv3_rec_infer/en_PP-OCRv3_rec_infer.tar


100%|██████████| 9.96M/9.96M [00:04<00:00, 2.13MiB/s]


download https://paddleocr.bj.bcebos.com/dygraph_v2.0/ch/ch_ppocr_mobile_v2.0_cls_infer.tar to /root/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer/ch_ppocr_mobile_v2.0_cls_infer.tar


100%|██████████| 2.19M/2.19M [00:07<00:00, 284kiB/s]

[2022/12/08 18:40:15] ppocr DEBUG: Namespace(alpha=1.0, benchmark=False, beta=1.0, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='/root/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_box_type='quad', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_limit_side_len=960, det_limit_type='max', det_model_dir='/root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_score_thresh=0.5, draw_img_save_dir='./inference_results', drop_score=0.5, e2e_algorithm='PGNet', e2e_char_dict_path='./ppocr/utils/ic15_dict.txt', e2e_limit_side_len=768, e2e_limit_type='max', e2e_model_dir=None, e2e_pgnet_mode='fast', e2e_pgnet_score_thresh=0.5, e2e_pgnet_valid_set

In [5]:
file  = '1.jpg'
img = cv2.imread(file)
img1 = cv2.imread(file)

In [6]:
#Identify Red colour boxes from Image
img_hsv=cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

# lower mask (0-10)
lower_red = np.array([0,50,50])
upper_red = np.array([10,255,255])
mask0 = cv2.inRange(img_hsv, lower_red, upper_red)

# upper mask (170-180)
lower_red = np.array([170,50,50])
upper_red = np.array([180,255,255])
mask1 = cv2.inRange(img_hsv, lower_red, upper_red)

# join my masks
mask = mask0+mask1

# set my output img to zero everywhere except my mask
output_img = img.copy()
output_img[np.where(mask==0)] = 0

In [8]:
cv2.imwrite('output.jpg',output_img)

True

In [31]:
data = []
json1 = {}
image = cv2.imread("output.jpg")
result = image.copy()
gray = cv2.cvtColor(output_img,cv2.COLOR_BGR2GRAY)

#_, threshold = cv2.threshold(gray, 0, 100, cv2.THRESH_BINARY)
ret,thresh_value = cv2.threshold(gray,0,255,cv2.THRESH_BINARY_INV)
kernel = np.ones((1,1),np.uint8)
dilated_value = cv2.dilate(thresh_value,kernel,iterations = 1)
cv2.imwrite('dilated_value.jpg',dilated_value)
contours, hierarchy = cv2.findContours(dilated_value,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
#index = 0
cordinates = []
for cnt in contours:   
    x,y,w,h = cv2.boundingRect(cnt)
    
    if (y > 1300) and (h > 5):
        cordinates.append((x,y,w,h))
        cropped = img1[y:y + h, x:x + w]
        #cv2.imwrite('x' + str(index) + '.jpg',cropped)
    
        # Apply paddle OCR on the cropped image
        result = ocr.ocr(cropped, det=False, cls=True)
        text = result[0][0][0]

        text = text.strip()
        text = text.replace('[','')
        text = text.replace(']','')
        if ':' in text:
            json1[text] = ''
        print(text)
        if ':' not in text:

            data.append(text)

        # Appending the text into file
        #index = index + 1

N:Plan/Job
N:Plan/Job
O:Sheet
O:Sheet
D:2
D1
s6123c11108
P:Revision
P:Revision
LAY oF o0ite
SHT.1OF9
F:Title 2
LAYOUT OF JET COOL #3
CMI EFCO, INC.,
CMI EFCO,INC.,
E:Title 1
E:Title 1
D:Vendor
D:Vendor


In [32]:
json1

{'N:Plan/Job': '',
 'O:Sheet': '',
 'D:2': '',
 'P:Revision': '',
 'F:Title 2': '',
 'E:Title 1': '',
 'D:Vendor': ''}

In [33]:
# remove empty values
test_list = [i for i in data if i]
#Assign values to keys
for k, v in json1.items():
    for i in test_list:

      if 'INC' in i and k == 'D:Vendor':
          json1[k] = i
      elif '#' in  i and  k =='E:Title 1':
          json1[k] = i
      elif 'sht' in i.lower() and k == 'F:Title 2':
          json1[k] = i
      elif 's' in i.lower() and k == 'N:Plan/Job':
          json1[k] = i
print(json1,)

{'N:Plan/Job': 'SHT.1OF9', 'O:Sheet': '', 'D:2': '', 'P:Revision': '', 'F:Title 2': 'SHT.1OF9', 'E:Title 1': 'LAYOUT OF JET COOL #3', 'D:Vendor': 'CMI EFCO,INC.,'}


In [35]:
output = json.dumps(json1)



In [22]:
json

{'[N:Plan/Job]': '',
 'N:Plan/Job': 'SHT.1OF9',
 'O:Sheet': '',
 '[D:2]': '',
 'P:Revision': '',
 'F:Title 2': 'SHT.1OF9',
 'E:Title 1': 'LAYOUT OF JET COOL #3',
 'D:Vendor': 'CMI EFCO,INC.,'}

In [36]:
output

'{"N:Plan/Job": "SHT.1OF9", "O:Sheet": "", "D:2": "", "P:Revision": "", "F:Title 2": "SHT.1OF9", "E:Title 1": "LAYOUT OF JET COOL #3", "D:Vendor": "CMI EFCO,INC.,"}'